実例で試すグラフアルゴリズムとグラフデータベース(Neo4j)ーー選挙の年なので政治家のtwitterでの関係を分析します

# 背景

フォローやフォロワー、ウェブサイトのリンク、アカウント間の関係、ユーザとレビューなど、繋がりを持つデータが増えていく一方であり、GoogleのPageRankに代表されるようにこれらの繋がりの中には莫大の価値が眠っています。一方で深層学習においても、グラフニューラルネットワークの発展が注目を集め、またBERTなどのトランスフォーマー模型も本質的には繋がり構造に着目することによって飛躍しています。グラフデータの扱いやグラフ構造に着目した解析が今後重要になっていく可能性が高いです。

# 記事について

ここでは具体例を使って、グラフデータベースとグラフアルゴリズムを紹介していきます。

グラフデータベースは[Neo4j](https://neo4j.com/)を使います。
言語はPythonです。

データについては、２０２０年９月現在、自民党総裁選とさらにそのあとの選挙が控えているので、政治家のTwitterアカウントのフォロー関係を使います。具体的には[Meyou](https://meyou.jp/group/category/politician/)から政治家・議員のアカウントを取得し、そのプロフィールとフォローしているアカウントを取得しました。データは９月６日から７日の間に収集したもので、全体は３５０弱アカウントで、フォローした人数が５０００以上を超えている３０ほどのアカウントについては５０００で足切りしています。
[こちら](https://drive.google.com/file/d/1IG903tly3AeGaVGdvqVJmZ9CqdgRpMV2/view?usp=sharing)からダウンロードできます。

# データ

次のようにデータを読み込みます

In [35]:
import json
import pandas as pd

from dateutil.parser import parse as date_parse

In [36]:
data_path = './data/neo4j_graph/politicians_200907.json'

In [37]:
with open(data_path, 'r') as f:
    data = json.load(f)

In [39]:
# 起点となるユーザのIDがキーで、フォロー対象のユーザIDリストが値、キーは政治アカウントで、対象はその限りではない
following_relationship = data[0]

# 政治アカウントのユーザ情報が入っています
user_information = data[1]

# アカウントが分類されたカテゴリ（政治家・議員のみ）とグループ（「自民党」「民進党」など）
category_information = data[2]